In [30]:
import pandas as pd
import time
import random
import twint # module for twitter scrapping
import nest_asyncio # module to solve notebook runtime issues. Twint would not work on a jupyter notebook without this

from textblob import TextBlob # simple module to perform a sentiment analysis

import nlp_bib as nlp # last project Collective Knowledge Extractor as a module


# Solve compatibility issues with notebooks and RunTime errors.
nest_asyncio.apply()

In [15]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]

# Function to get sentiment 
def apply_blob(sentence):
    temp = TextBlob(sentence)
    
    if temp.detect_language() != 'en':
        frase = TextBlob(str(temp.translate(to='en')))
    else:
        frase = temp
    
    polarity = frase.sentiment[0]
    sent = 'Neutral'
    if polarity >= 0.35:
        sent = 'Positive'
    elif polarity <= -0.35:
        sent = 'Negative'
    
    return polarity, sent

def analyze_tweets(term_to_search, limit=200):
    c = twint.Config()
    c.Search = term_to_search
    c.Limit = limit
    c.Pandas = True
    with HiddenPrints():
        twint.run.Search(c)
    df_pd = twint_to_pandas(["date", "username", "tweet", "hashtags", "nlikes"])
    return df_pd



def extract_knowledge(data, startwords, n_words=4, n_sentences=10):
    ecc = nlp.CollectiveKnowledgeExtractor('portuguese')

    tweets = ''
    for text in data['tweet']:
        tweets += text + '. '

    in_wd = ecc.aux_preprocess(startwords)
    try:
        selected_sentences = ecc.full_extraction(tweets, n_words, n_sentences, in_wd)
        return selected_sentences
    except:
        return None

In [16]:
tw_df = analyze_tweets('bolsonaro', limit=10000)

tw_df

,date,username,tweet,hashtags,nlikes
0,2021-07-04 22:06:01,Chantecler,@CNNBrasil Eduardo Paes não tem credibilidade ...,[],0
1,2021-07-04 22:06:01,olavombarros,@Sihdney @jairbolsonaro Vc devia tomar vergonh...,[],0
2,2021-07-04 22:06:01,Igor_Prezado,@jairbolsonaro #BolsonaroAte2026 🇧🇷 Vai ter v...,[bolsonaroate2026],0
3,2021-07-04 22:06:00,Jocenilda1,@Lfmacielmg Bolsonaro carrega mundo a fora o t...,[ladraodevacina],0
4,2021-07-04 22:06:00,lelemmk,@afroputoo @jairbolsonaro sabe o que é ironia ...,[],0
...,...,...,...,...,...
9995,2021-07-04 19:53:29,FelipeRamosSil6,@Debora57705883 @ttskaue @Jotacesse @re_doming...,[],0
9996,2021-07-04 19:53:27,Povodemerda5,"BOLSONARO É ASSASSINO , GENOCÍDA E CORRRUPTO ...","[ladraodevacina, forabol卐onarogenocida, bolson...",0
9997,2021-07-04 19:53:26,foursurvivors,O povo sai nas ruas Fora Bolsonaro Fora Geno...,"[forabolsonaro, forabolsonarogenocida, 3jforab...",0
9998,2021-07-04 19:53:25,Alvesalces,@PaulaKharine @KingAbdullahII @QueenRania @sos...,[],0


In [17]:
selected_sentences = extract_knowledge(tw_df, ['impeachment', 'genocida', 'manifestação'], n_words=2, n_sentences=40)

In [31]:
random.sample(select_sentences, k=4)

['@jairbolsonaro @henriquelorezo Se fosse uma manifestação descente ninguém seria atingido.',
 'Às ruas foram tomadas por milhares de manifestantes pedindo o impeachment do atual presidente genocida!',
 '@jairbolsonaro Bozo genocida !!!!',
 '@freire_roberto Bolsonaro é um ladrão genocida.. @senadorhumberto Por que vcs nao apuram a corrupção e também a corrupção do governo bolsonaro?']

In [32]:
for sentence in selected_sentences:
    time.sleep(1)
    sent = apply_blob(sentence)
    if sent[1] != 'Neutral':
        print(sent)
        print(sentence)

(0.48828125, 'Positive')
Eu acho engraçado quando eu vejo uma pessoa com a camisa do che guevara, gritando Bolsonaro genocida.. @taliriapetrone @marinairisvoz @TeresaCristina IMPEACHMENT URGENTE!!!
(0.39285714285714285, 'Positive')
Em partes o Lula não quer que tenha impeachment do bolsonaro né, pq ele sabe que se o Bolsonaro sair e não disputar as eleições, as chances de dar um Ciro ou outro são maiores.
(0.5, 'Positive')
@jairbolsonaro mais de 520 mil mortes de brasileiros inocentes e tu seu VERME GENOCIDA MALDITO debochando teu lugar e da tua família e tua quadrilha vai ser na CADEIA NA SELA MAIS IMUNDA  #LadrãodeVacina.
(0.5, 'Positive')
@jairbolsonaro mais de 520 mil mortes de brasileiros inocentes e tu seu VERME GENOCIDA MALDITO debochando teu lugar e da tua família e tua quadrilha vai ser na CADEIA NA SELA MAIS IMUNDA  #LadrãodeVacina.
(-0.5, 'Negative')
Imaginem que os caras que cansaram de xingar Bolsonaro de genocida dizendo que ele não queria vacinas, agora o chamam de ladrã